In [ ]:


import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import random
%matplotlib inline


def split_dataset (dataset ,split_ratio): # numpy
    training_set = []
    training_size = int(len(dataset)*split_ratio)
    temp_test_dataset = list(dataset)
    random_row_number = 0
    random_row = []
    while (len(training_set)<training_size):
        random_row_number = random.randrange(len(temp_test_dataset))
        random_row = temp_test_dataset.pop(random_row_number)
        training_set.append(random_row)
    test_dataset = temp_test_dataset
    return [training_set, test_dataset]


def mean(numbers):
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)


def categorize_into_classes (dataset):
    list_of_classes = []
    list_of_classes = np.unique(dataset[:,-1])
    classes_features_dict = {}
    
    for i in list_of_classes:
        data_for_the_class =  dataset[dataset[:,-1]==i].copy()
        data_for_the_class = data_for_the_class[:,:-1].copy() # cutting off the last column
        classes_features_dict[i] = data_for_the_class
    return classes_features_dict




def generate_stats(dataset):
    temp_mean = 0
    std_dev = 0
    stats_array = []
    # dataset = dataset[:,:-1].copy() # removed the last column
    index = 0
    for index in range(np.size(dataset,1)):
        temp_mean = mean(dataset[:,index])
        std_dev = stdev(dataset[:,index])
        if (std_dev==0):
            std_dev = 0.0000022
        stats_array.append([temp_mean,std_dev])
        
    return stats_array

def generate_stats_by_class(dataset):

    no_of_features = dataset[1].shape[0]
    classes_features_dict = categorize_into_classes (dataset)
    class_wise_stats = {}
    for class_value, class_data in classes_features_dict.items():
        class_wise_stats[class_value] = generate_stats(class_data)

    cov_matrix = np.cov(dataset[:,:-1],rowvar=False)+0.2*np.identity(no_of_features-1)
    return (class_wise_stats,cov_matrix)

def calculate_probability(x, mean_vector, cov_matrix):
    
   
    cov_matrix_inv = np.linalg.inv(cov_matrix)
    cov_matrix_det = np.linalg.det(cov_matrix)
    
    no_of_features = x.shape[0]
    
    factor = 1/(math.sqrt(math.pow((2 * math.pi),no_of_features)*cov_matrix_det))
    firstfactor = (x-mean_vector).T
    secondfactor = (x-mean_vector)
    
    temp = (1/2) *firstfactor@cov_matrix_inv@secondfactor
    exponent = math.exp(-temp)
    
    return (factor) * exponent


def calculate_all_probabilities(class_wise_stats,cov_matrix, input_vector):
    
    mean_dict = {}

    for class_value, class_data in class_wise_stats.items():
        for i in range(len(class_data)):
            if class_value not in mean_dict:
                mean_dict[class_value] = []
            mean_dict[class_value].append(class_data[i][0]) # i th row is the ith feature
    
    
    # mean_dict
    probabilities = {}
    for class_value,class_data in mean_dict.items():
        mean_vector= mean_dict[class_value]        
        x = input_vector
        probabilities[class_value] = calculate_probability (x, mean_vector, cov_matrix)
    
    return probabilities


def predict(class_wise_stats,cov_matrix, input_vector):
    probabilities = calculate_all_probabilities(class_wise_stats,cov_matrix, input_vector)
    label = None
    prob = -1
    
    for class_value, probability in probabilities.items():
        if label is None or probability > prob:
            prob = probability
            label = class_value
    return label


def get_predictions(class_wise_stats,cov_matrix, test_dataset):
    predictions = []
    for i in range(len(test_dataset)):
        result = predict(class_wise_stats,cov_matrix,test_dataset[i])
        predictions.append(result)
    return predictions

def getAccuracy(test_dataset, predictions):
    correct = 0
    for i in range(len(test_dataset)):
        if test_dataset[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(test_dataset))) * 100.0

orig_dataset = pd.read_csv("RainInAustralia.csv")

new_dataset = np.array(orig_dataset)

split_ratio = 0.75

training_set, test_dataset = split_dataset(new_dataset, split_ratio)

training_set = np.array(training_set)
test_dataset = np.array(test_dataset)


parameters,cov_matrix = generate_stats_by_class(training_set)



predictions = get_predictions(parameters,cov_matrix, test_dataset[:,:-1])

getAccuracy(test_dataset, predictions)

